In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

import json
import phoenix as px
from llama_index.core import (
    Settings,
    set_global_handler,
)
import nest_asyncio
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import (
    get_qa_with_reference,
    get_retrieved_documents,
)
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from tqdm import tqdm

nest_asyncio.apply()

In [2]:
session = px.launch_app()
set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
llm = Ollama(model="phi4", request_timeout=60.0)
embedder = OllamaEmbedding(
    model_name="nomic-embed-text",
)

Settings.llm = llm
Settings.embed_model = embedder

In [4]:
llm.complete("hello")

CompletionResponse(text='Hello! How can I assist you today? If you have any questions or need information, feel free to let me know. 😊', additional_kwargs={'tool_calls': []}, raw={'model': 'phi4', 'created_at': '2025-01-22T10:59:40.355508Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13384279000, 'load_duration': 558924584, 'prompt_eval_count': 11, 'prompt_eval_duration': 10879000000, 'eval_count': 28, 'eval_duration': 1943000000, 'message': Message(role='assistant', content='Hello! How can I assist you today? If you have any questions or need information, feel free to let me know. 😊', images=None, tool_calls=None), 'usage': {'prompt_tokens': 11, 'completion_tokens': 28, 'total_tokens': 39}}, logprobs=None, delta=None)

In [5]:
embedder.get_text_embedding("hello")

[0.42144137620925903,
 -0.13775025308132172,
 -4.1203837394714355,
 -0.3233436644077301,
 0.8000353574752808,
 1.0508289337158203,
 0.2911536395549774,
 -0.06047961860895157,
 -0.3482259511947632,
 -0.9226974248886108,
 -0.21647962927818298,
 1.2151598930358887,
 1.3565224409103394,
 1.343890905380249,
 1.057612419128418,
 -1.2590347528457642,
 0.6766302585601807,
 -1.1090795993804932,
 -0.9175370335578918,
 0.632752001285553,
 0.21995720267295837,
 -1.5684449672698975,
 0.11625935137271881,
 -0.13814355432987213,
 4.047388076782227,
 -0.10346291959285736,
 0.4391018748283386,
 1.9591352939605713,
 0.03767038881778717,
 -0.5290690660476685,
 0.4780785143375397,
 -0.440297394990921,
 0.3601878881454468,
 0.07514747977256775,
 0.6475369930267334,
 -0.44028440117836,
 -0.08981560915708542,
 0.2652130722999573,
 0.3800978362560272,
 0.45303770899772644,
 -0.4143969416618347,
 -0.22600185871124268,
 0.21730023622512817,
 -0.6651782393455505,
 1.207326054573059,
 0.1712249219417572,
 -0.5195

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [7]:
documents = SimpleDirectoryReader(input_files=["data/Employee Communication Policy v1.pdf"]).load_data()

In [8]:
index = VectorStoreIndex.from_documents(documents, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=3)

In [9]:
response = query_engine.query("What does the term 'Social Media' refer to in the document?")
print(response)

In the document, "social media" refers to online platforms where employees can communicate and share content. These platforms allow for interactions such as posting messages, comments, sharing or endorsing information, and following other accounts. The policy outlines guidelines on how employees should conduct themselves respectfully and thoughtfully while engaging with social media, including managing responses to criticism and deciding whether to follow colleagues' accounts. It also provides provisions for handling aggressive or abusive behavior encountered on these platforms.


In [10]:
for source in response.source_nodes:
    print(source, end="\n")

Node ID: 1b3540e6-3d93-44d9-be37-6dc89cb901c8
Text: EMPLOYEE  COMMUNICATIONS  POLICY    Document  owner:
Reference:    Author:   Communications  and  Human  Resources   Date:
14/08/2024   Approved  By:    Head  Human  Resources    Approval
Date:   14/01/2025   Classification:   Internal   Version:  1.0
Effective  Date:  14/01/2025    Record  of  Change   Name  Designation
Reason  for  Ch...
Score:  0.655

Node ID: d3b0de25-0acc-45f7-af3e-3a1688fed8b3
Text: 1.2.5  Members  of  employees  are  to  always  treat  others
with  respect  on  social  media.  If  someone  questions  or
criticizes   the   work   we   do   at   Helium   Health,   responses
should   be   thoughtful   and   kind.   If   the   criticism   is
especially   aggressive   or   inconsiderate,   it   is   recommended
that   ...
Score:  0.646

Node ID: e993c6ae-c7bb-4190-a2ec-e27ff782a289
Text:
Score:  0.631



In [11]:
response = query_engine.query("List some social media guidelines for usage?")
print(response)

for source in response.source_nodes:
    print(source, end="\n")

The organization encourages team members to act as ambassadors while using social media, emphasizing that personal expression should be balanced with responsibility towards customers, partners, and external parties. Employees may choose whether or not to publicly associate themselves with the company on social media platforms (excluding LinkedIn), but they must take responsibility for their content. Posting or sharing inflammatory material, inciting violence, spreading misinformation, or anything lacking basic human decency is prohibited.

Team members should always treat others with respect on social media, responding thoughtfully and kindly to criticism about the organization’s work. In cases of aggressive or inconsiderate criticism, it's recommended that employees refrain from responding. Additionally, team members have the right to mute or block individuals who are threatening or abusive on these platforms.

The company has official accounts managed by authorized personnel for publ

In [12]:
response = query_engine.query("What are professional websites?")
print(response)

for source in response.source_nodes:
    print(source, end="\n")

Professional websites refer to online platforms that maintain a formal tone and appearance, often used by businesses, organizations, or individuals for official purposes. These sites typically include features such as informative content, contact details, services offered, and company news. The design is usually clean and structured to convey professionalism and credibility. Additionally, they may incorporate multimedia elements like images and videos while adhering to best practices in usability and accessibility.

In the context of a corporate environment, professional websites should align with organizational branding guidelines, including approved logos and color schemes. Email signatures associated with such sites also need to project this professional image by using standardized formats that reflect the organization's values and identity. 

Ultimately, these websites serve as digital representations of an entity’s brand, aiming to engage effectively with their audience while upho

## Document Splitters

In [13]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=10,
)

print(f"Len documents: ", len(documents))
nodes = splitter.get_nodes_from_documents(documents)
print(f"Len nodes: ", len(nodes))

Len documents:  7
Len nodes:  25


In [14]:
index = VectorStoreIndex(nodes=nodes, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

In [15]:
response = query_engine.query("What does the term 'Social Media' refer to in the document?")
print(response, end="\n")

for source in response.source_nodes:
    print(source, end="\n")

The term "Social Media" in the document refers to online platforms that facilitate interactions among people. These platforms include well-known networks like Twitter (formerly X), Facebook, TikTok, Snapchat, and others of similar nature where individuals engage primarily for creating or curating content. Importantly, these are distinguished from professional websites.
Node ID: 3f57a4fc-d73a-4e7e-a3a6-65b46f7ce429
Text: Definitions  1.1.1.  The  usage  of  the  term  “Social  Media”
now  and  hitherto  refer  to  online  platforms  which  aid
interactions   among   people   including,   amongst   others,   X
(formerly   twitter),   Facebook,   Tiktok,   Snapchat   or   any
other   platform   by   which   individuals   engage   for   the
purpose(s)   of ...
Score:  0.808

Node ID: f39256b1-a5a1-487c-9385-74adde139602
Text: If   you   provide   us   with   any   contact   information
for   customer   service   purposes,   it   will   only   be   used
to   manage   your   enquiry   and   

In [16]:
response = query_engine.query("List some social media guidelines for usage?")
print(response, end="\n\n")

for source in response.source_nodes:
    print(source, end="\n")

Employees are encouraged to act as ambassadors for the company on social media. It's important that they consider the impact of their online engagements, especially because these actions may be viewed by customers, partners, and other external parties.

Only authorized employees have permission to respond to public queries via official company accounts. Employees must confirm the authenticity of any news about the company before sharing, resharing, or endorsing it on social media platforms; they should consult line managers or members of the Communications team for verification.

Additionally, if a social media user intends to share personal data with the company's accounts, administrators are advised to suggest using direct messaging instead. If personal data is shared publicly by mistake, administrators should attempt to delete it or encourage the user to use private messaging to prevent potential misuse.

Node ID: 3f57a4fc-d73a-4e7e-a3a6-65b46f7ce429
Text: Definitions  1.1.1.  The  

In [17]:
response = query_engine.query("What are professional websites?")
print(response, end="\n\n")

for source in response.source_nodes:
    print(source, end="\n")

Professional websites refer to platforms designed for networking and sharing content related to one's career or work achievements. These include sites like LinkedIn, AngelList, Crunchbase, among others. They serve as spaces where individuals can represent their professional identities, share their experiences, and connect with peers within their industry.

These platforms have specific expectations regarding how an individual associated with a company should present themselves. This includes maintaining accurate profiles that reflect one's official title, term of employment, and using professional images like headshots. The language used on these sites should be considered carefully, especially when addressing sensitive topics, to ensure it is thoughtful rather than blunt or inciting.

Furthermore, while employees are permitted to identify with their employer publicly on platforms other than LinkedIn, they must still adhere to guidelines ensuring that any content shared does not spread

## Evaluation

In [18]:
from llama_index.llms.lmstudio import LMStudio

In [19]:
from phoenix.evals import LiteLLMModel
from litellm import completion

/Users/sasuusen/miniconda3/envs/Helium/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [20]:
import os

os.environ['LM_STUDIO_API_BASE'] = "http://127.0.0.1:1234/v1"
os.environ['LM_STUDIO_API_KEY'] = "sk-123"

In [21]:
eval_model = LiteLLMModel(model="lm_studio/phi4-latest", _verbose=True)

In [22]:
response = eval_model("Hey there, what is 2+2?")
print(str(response))

2 + 2 equals 4. If you have any more questions or need further assistance, feel free to ask!


In [23]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

In [24]:
queries_df

,input,output,reference
context.span_id,,,
78c28db07ff9886c,What does the term 'Social Media' refer to in ...,"In the document, ""social media"" refers to onli...",EMPLOYEE COMMUNICATIONS POLICY Document ...
2afccf2582b4cae4,List some social media guidelines for usage?,The organization encourages team members to ac...,1.2.5 Members of employees are to always...
b886d4efd22d34fb,What are professional websites?,Professional websites refer to online platform...,1.2.5 Members of employees are to always...
35fab9bee5eb6300,What does the term 'Social Media' refer to in ...,"The term ""Social Media"" in the document refers...",Definitions 1.1.1. The usage of the term...
3354bc4371077c9f,List some social media guidelines for usage?,Employees are encouraged to act as ambassadors...,Definitions 1.1.1. The usage of the term...
1e5c387c5d636d65,What are professional websites?,Professional websites refer to platforms desig...,Definitions 2.1.1. The usage of the term...


In [25]:
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
530af4522d6d8273 0                  d51ccb08c0fedf13cb7faadc0ef8709b   
                 1                  d51ccb08c0fedf13cb7faadc0ef8709b   
                 2                  d51ccb08c0fedf13cb7faadc0ef8709b   
fbefcb4215e8cdc8 0                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
                 1                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
                 2                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
ab83ed62670696ff 0                  9d6a962b94adafa8b989b7b5939b443c   
                 1                  9d6a962b94adafa8b989b7b5939b443c   
                 2                  9d6a962b94adafa8b989b7b5939b443c   
fed4ecdba15db354 0                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 1                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 2                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 3                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 4                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
aa15c5e3b65359af 0                  de3849188cb84f02bf780b556c9b02eb   
                 1                  de3849188cb84f02bf780b556c9b02eb   
                 2                  de3849188cb84f02bf780b556c9b02eb   
                 3                  de3849188cb84f02bf780b556c9b02eb   
                 4                  de3849188cb84f02bf780b556c9b02eb   
a9e42c0917f8a29f 0                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 1                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 2                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 3                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 4                  1ef2fc386563f4bfb7f2cccc42a93edd   

                                                                                input  \
context.span_id  document_position                                                      
530af4522d6d8273 0                  What does the term 'Social Media' refer to in ...   
                 1                  What does the term 'Social Media' refer to in ...   
                 2                  What does the term 'Social Media' refer to in ...   
fbefcb4215e8cdc8 0                       List some social media guidelines for usage?   
                 1                       List some social media guidelines for usage?   
                 2                       List some social media guidelines for usage?   
ab83ed62670696ff 0                                    What are professional websites?   
                 1                                    What are professional websites?   
                 2                                    What are professional websites?   
fed4ecdba15db354 0                  What does the term 'Social Media' refer to in ...   
                 1                  What does the term 'Social Media' refer to in ...   
                 2                  What does the term 'Social Media' refer to in ...   
                 3                  What does the term 'Social Media' refer to in ...   
                 4                  What does the term 'Social Media' refer to in ...   
aa15c5e3b65359af 0                       List some social media guidelines for usage?   
                 1                       List some social media guidelines for usage?   
                 2                       List some social media guidelines for usage?   
                 3                       List some social media guidelines for usage?   
                 4                       List some social media guidelines for usage?   
a9e42c0917f8a29f 0                                    What are professional websites?   
                 1                                    What are professional websites?   
                 2                                    What are professional websites?   
                 3                         

In [26]:
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

In [27]:
%%time
hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
    # concurrency=1
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
    # concurrency=1
)[0]

I0000 00:00:1737543835.687621    9601 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


run_evals |          | 0/12 (0.0%) | ⏳ 00:00<? | ?it/s


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.
Requeuing...
Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.
Requeuing...
Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Er

run_evals |          | 0/24 (0.0%) | ⏳ 00:00<? | ?it/s


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 1: raised litellm.BadRequestError: Lm_studioException - Error code: 400 - {'error': 'The model has crashed without additional information. (Exit code: 11)'}
Requeuing...
CPU times: user 20.5 s, sys: 4.17 s, total: 24.7 s
Wall time: 51min 15s


In [28]:
px.Client().log_evaluations(
    SpanEvaluations(
        eval_name="Hallucination", dataframe=hallucination_eval_df
    ),
    SpanEvaluations(
        eval_name="QA Correctness", dataframe=qa_correctness_eval_df
    ),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)